# Outcome Prediction with RGAT Model

## 1. Load data from "processed_data" folder

In [1]:
import time
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import RGATConv
from torch_geometric.utils import to_undirected

In [12]:
num_patients = 1000
inverse_triples = True
embed_dim = 32
num_heads = 2

entity = pd.read_csv(f'processed_data/sphn_entities_{num_patients}_noOutcome.tsv', sep='\t', index_col=0, header=None)
entity = entity.to_dict()[1]

indices = []
for i in range(num_patients):
    idx = f'<http://nvasc.org/synth_patient_{i}>'
    indices.append(entity[idx])

events = pd.read_csv(f'processed_data/sphn_events_{num_patients}_noOutcome.tsv', sep='\t', header=None)
y = joblib.load(f'../Data Generation/outcomes_{num_patients}_0.joblib')

skf = StratifiedKFold(n_splits=5)
train_idx, test_idx = next(iter(skf.split(indices, y)))
indices = np.asarray(indices)
y = np.asarray(y)

edge_index = torch.vstack((torch.Tensor(events[0]).long(),torch.Tensor(events[2]).long()))
edge_type = torch.Tensor(events[1]).long()
train_x = torch.Tensor(indices[train_idx]).long()
train_y = torch.Tensor(y[train_idx]).long()
test_x = torch.Tensor(indices[test_idx]).long()
test_y = torch.Tensor(y[test_idx]).long()
num_nodes = len(entity)

if inverse_triples == True:
    edge_index = to_undirected(edge_index)
    edge_type = torch.cat((edge_type, edge_type))

data = Data(
    edge_index=edge_index,
    edge_type=edge_type,
    train_idx=train_x,
    train_y=train_y,
    test_idx=test_x,
    test_y=test_y,
    num_nodes=num_nodes,
)
embedding = torch.nn.Embedding(data.num_nodes, embed_dim)
data.x = embedding.weight
data.num_relations = data.num_edge_types
data.num_classes = 3
print(data)

Data(edge_index=[2, 221870], edge_type=[221870], train_idx=[800], train_y=[800], test_idx=[200], test_y=[200], num_nodes=33563, x=[33563, 32], num_relations=10, num_classes=3)


In [5]:
pd.DataFrame(y).value_counts()

0
0    456
1    429
2    115
Name: count, dtype: int64

## 2. Training the model

In [13]:
class RGAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels,
                 num_relations):
        super().__init__()
        self.conv1 = RGATConv(in_channels, hidden_channels, num_relations, heads=num_heads)
        self.conv2 = RGATConv(hidden_channels*num_heads, hidden_channels, num_relations, heads=num_heads)
        self.lin = torch.nn.Linear(hidden_channels*num_heads, out_channels)

    def forward(self, x, edge_index, edge_type):
        x = self.conv1(x, edge_index, edge_type)
        x = F.leaky_relu(x)
        x = F.dropout(x, p=0.2)
        x = self.conv2(x, edge_index, edge_type)
        x = F.leaky_relu(x)
        # x = F.dropout(x, p=0.3)
        x = self.lin(x)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

num_classes = 3
num_relations = data.num_edge_types
model = RGAT(embed_dim, 32, num_classes, num_relations).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0005)
# class_weight = torch.Tensor([456/456, 456/429, 456/115]).to(device)

In [14]:
def train():
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_type)
    loss = F.nll_loss(out[data.train_idx], data.train_y, 
                    #   weight=class_weight,
                      )
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    pred = model(data.x, data.edge_index, data.edge_type).argmax(dim=-1)
    train_acc = float((pred[data.train_idx] == data.train_y).float().mean())
    test_acc = float((pred[data.test_idx] == data.test_y).float().mean())
    return train_acc, test_acc

In [15]:
times = []
for epoch in range(1, 101):
    start = time.time()
    loss = train()
    if epoch % 5 ==0:
        train_acc, test_acc = test()
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f} '
            f'Test: {test_acc:.4f}')
    times.append(time.time() - start)
print(f"Median time per epoch: {torch.tensor(times).median():.4f}s")

Epoch: 05, Loss: 0.9609, Train: 0.5125 Test: 0.4600
Epoch: 10, Loss: 0.8645, Train: 0.6087 Test: 0.4300
Epoch: 15, Loss: 0.6538, Train: 0.7113 Test: 0.4450
Epoch: 20, Loss: 0.4313, Train: 0.8162 Test: 0.4700
Epoch: 25, Loss: 0.2810, Train: 0.8975 Test: 0.4650
Epoch: 30, Loss: 0.1960, Train: 0.9262 Test: 0.4300
Epoch: 35, Loss: 0.1781, Train: 0.9575 Test: 0.4600
Epoch: 40, Loss: 0.1375, Train: 0.9575 Test: 0.4250
Epoch: 45, Loss: 0.0988, Train: 0.9650 Test: 0.4600
Epoch: 50, Loss: 0.0710, Train: 0.9825 Test: 0.3900
Epoch: 55, Loss: 0.0518, Train: 0.9875 Test: 0.4250
Epoch: 60, Loss: 0.0383, Train: 0.9937 Test: 0.4400
Epoch: 65, Loss: 0.0404, Train: 0.9825 Test: 0.4600
Epoch: 70, Loss: 0.0441, Train: 0.9825 Test: 0.4700
Epoch: 75, Loss: 0.0388, Train: 0.9925 Test: 0.4350
Epoch: 80, Loss: 0.0287, Train: 0.9925 Test: 0.4200
Epoch: 85, Loss: 0.0315, Train: 0.9925 Test: 0.4450
Epoch: 90, Loss: 0.0241, Train: 0.9950 Test: 0.4350
Epoch: 95, Loss: 0.0211, Train: 0.9975 Test: 0.4850
Epoch: 100, 

In [16]:
from sklearn.metrics import roc_auc_score

model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index, data.edge_type).cpu()
    prob = F.softmax(out, dim=1)
auc = roc_auc_score(data.test_y.cpu(), prob[data.test_idx.cpu()], multi_class='ovr')
print(f'ROC AUC score: {auc:.4f}')
# print(prob.shape)

ROC AUC score: 0.5534
